In [ ]:
import torch.utils.data

from utils import *

In [ ]:
config = Config().load(os.path.join("configs", "config.json"))

In [ ]:
class MultiCrossEntropy(nn.Module):
    def __init__(self):
        super().__init__()

        self.entropy = nn.CrossEntropyLoss()

    def forward(self, yPred, yTrue, mask):
        loss = 0
        for key in yPred:
            target = yTrue[key]
            target[mask] = -100
            loss += self.entropy(yPred[key], target)

        return loss / len(yPred)

In [ ]:
def iterate(dataset):
    while True:
        for batch in dataset:
            yield batch

def trainModel(config, modelClass, datasetClass, objective):
    model = modelClass(config.model)
    dataset = datasetClass(config.dataset)

    train, test = torch.utils.data.random_split(dataset, [0.8, 0.2])
    train = DataLoader(train, batch_size=config.batchSize, shuffle=True, collate_fn=datasetClass.collate)
    test = DataLoader(test, batch_size=config.batchSize, shuffle=True, collate_fn=datasetClass.collate)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learningRate)

    client = Client("127.0.0.1", 12945)

    testIter = iterate(test)

    for batch in train:


